In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **0. Import Packages**

In [ ]:
import pandas as pd

## **1. Read in Data**

In [ ]:
name_col = ['ncodpers', 'fecha_dato', 'ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1',
           'ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
           'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1',
           'ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1',
           'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1',
           'ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
           'ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1',
           'ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1']

reader = pd.read_csv("/content/drive/MyDrive/BT5153Applied Machine Learning/Data/train_ver2.csv",chunksize=1e6, usecols=name_col)
df_train = pd.concat([chunk for chunk in reader])

df_train.shape

(13647309, 26)

In [ ]:
df_train.groupby(['fecha_dato']).size()

fecha_dato
2015-01-28    625457
2015-02-28    627394
2015-03-28    629209
2015-04-28    630367
2015-05-28    631957
2015-06-28    632110
2015-07-28    829817
2015-08-28    843201
2015-09-28    865440
2015-10-28    892251
2015-11-28    906109
2015-12-28    912021
2016-01-28    916269
2016-02-28    920904
2016-03-28    925076
2016-04-28    928274
2016-05-28    931453
dtype: int64

## **2. Slice data from 2015-11-28 to 2016-04-28**

In [ ]:
df_slice = df_train[(df_train["fecha_dato"]>="2015-11-28") & (df_train["fecha_dato"]<"2016-05-28")]
df_slice.groupby(['fecha_dato']).size()

fecha_dato
2015-11-28    906109
2015-12-28    912021
2016-01-28    916269
2016-02-28    920904
2016-03-28    925076
2016-04-28    928274
dtype: int64

## **3. Groupby DateMth and Sum Up**

In [ ]:
df_groupby = df_slice.groupby(['ncodpers'], as_index = False).sum(['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1',
           'ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
           'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1',
           'ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1',
           'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1',
           'ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
           'ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1',
           'ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1'])

## **4. Delete Blank Rows**

In [ ]:
len_with_blank = len(df_groupby)
df_groupby_without_blank = df_groupby[df_groupby.iloc[:,1:].sum(axis=1) != 0]
print(len_with_blank, len(df_groupby_without_blank))

938442 706652


## **5. Check The Distribution**

In [ ]:
df_count = pd.DataFrame(df_groupby_without_blank.iloc[:,1:].astype(bool).sum(axis=1), columns=["count of product rates"])
df_count.groupby(['count of product rates']).size()

count of product rates
1     439350
2     123751
3      50811
4      33537
5      22676
6      15706
7      10627
8       5944
9       2692
10      1061
11       381
12        87
13        26
14         2
15         1
dtype: int64

## **6. Save**

In [ ]:
df_groupby_without_blank.columns = ['Cust_ID', 'Savings_Acct', 'Guarantees', 'Cur_Acct', 'Derivative_Acct', 'Payroll_Acct',
                'Junior_Acct', 'Mas_Particular_Acct', 'Particular_Acct', 'Particular_Plus_Acct', 'Short_Term_Deposits',
                'Med_Term_Deposits', 'Long_Term_Deposits', 'e-Acct', 'Funds', 'Mortgage', 'Pension1', 'Loans',
                'Taxes', 'Credit_Card', 'Securities', 'Home_Acct', 'Payroll', 'Pension2', 'Direct_Debit']
df_groupby_without_blank.to_csv("/content/drive/MyDrive/BT5153Applied Machine Learning/Data/train_data_autorec.csv", index = False)

## **7. Test Data Ground Truth**


In [ ]:
df_slice = df_train[(df_train["fecha_dato"]=="2016-05-28")]
df_test = df_slice[df_slice.iloc[:,2:].sum(axis=1) != 0] # delete blank rows
df_test = df_test.iloc[:,1:]
df_test.columns = ['Cust_ID', 'Savings_Acct', 'Guarantees', 'Cur_Acct', 'Derivative_Acct', 'Payroll_Acct',
                'Junior_Acct', 'Mas_Particular_Acct', 'Particular_Acct', 'Particular_Plus_Acct', 'Short_Term_Deposits',
                'Med_Term_Deposits', 'Long_Term_Deposits', 'e-Acct', 'Funds', 'Mortgage', 'Pension1', 'Loans',
                'Taxes', 'Credit_Card', 'Securities', 'Home_Acct', 'Payroll', 'Pension2', 'Direct_Debit']
df_test.to_csv("/content/drive/MyDrive/BT5153Applied Machine Learning/Data/test_data_ground_truth_autorec.csv", index = False)